In [1]:
import re

import pandas as pd

import torch
import torchtext
# from torchtext.datasets import text_classification
import os
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data.dataset import random_split
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

from torch.utils.data import  Dataset

from nltk.corpus import stopwords

In [2]:
df_train = pd.read_csv("../../data/train.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
df_valid = pd.read_csv("../../data/validation.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)
df_test = pd.read_csv("../../data/test.csv", header=None, names=["quadrant", "lyrics"], skiprows=1)

df_train = pd.concat([df_train, df_valid])

In [3]:
df_train.lyrics.iloc[776]

"Life inside the music box ain't easy\nThe mallets hit, the gears are always turning\nAnd everyone inside the mechanism\nIs yearning to get out\n\nAnd sing another melody completely\nSo different from the one they're always singing\nI close my eyes and think that I have found me\nBut then I feel mortality surround me\n\nI want to sing another melody\nSo different from the one I always sing"

In [4]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [5]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_preprocessing(text):
    text = text.lower() # lower text
    text = replace_contractions(text) # remove contactions
    text = "".join("".join(text).replace("\n", " ").replace("\r", " ")) # remove \n and \r
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace symbols with space
    text = BAD_SYMBOLS_RE.sub('', text) # replace bad characters with nothing
    text = " ".join([word for word in text.split() if word not in STOPWORDS]) # remove stopwords
    
    return text

In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text_preprocessing(text))

vocab = build_vocab_from_iterator(yield_tokens(df_train.lyrics.values), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

print(f"Vocab Length: {len(vocab)}")

Vocab Length: 82629


In [7]:
text_preprocessing(df_train.lyrics.iloc[776])

'life inside music box easy mallets hit gears always turning everyone inside mechanism yearning get sing another melody completely different one always singing close eyes think found feel mortality surround want sing another melody different one always sing'

In [8]:
class LyricsDataset(Dataset):
    def __init__(self, dataframe):

        self.data = dataframe

    def __getitem__(self, idx):

        lyrics = text_preprocessing(self.data.lyrics.iloc[idx])
        labels = int(self.data.quadrant.iloc[idx])

        return labels, lyrics

    def __len__(self):
        return len(self.data)

In [9]:
train_dataset = LyricsDataset(df_train)
test_dataset = LyricsDataset(df_test)

In [10]:
# for idx, data in enumerate(train_dataset):
#     print(idx)

In [11]:
NGRAMS = 3
BATCH_SIZE = 64

In [12]:
def generate_batch(batch):
    label = torch.tensor([label_pipeline(entry[0]) for entry in batch])
    print(label)
    text = [torch.tensor(text_pipeline(entry[1]), dtype=torch.int64) for entry in batch]
    print(text)
    offsets = [0] + [entry.size(0) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    print(offsets)
    text = torch.cat(text)
    return text, offsets, label

data = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=generate_batch)

text, offseets, label = next(iter(data))

tensor([1, 1, 0, 0, 3, 0, 1, 1, 0, 1, 0, 1, 2, 1, 3, 0, 3, 3, 2, 2, 2, 1, 2, 1,
        2, 2, 1, 0, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 2, 1, 2, 3, 3, 0, 2, 0, 0, 0,
        0, 3, 0, 3, 0, 2, 1, 2, 0, 0, 0, 1, 3, 3, 0, 0])
[tensor([ 2444,    90,   888,  2444,    90,   888,   141, 14439,     8,   313,
          360,  8688,   867,  2060,   333,   127,   608,   133, 14470,  2312,
         9943,  6510,    73,   687,    39,  1728,   137,    18, 23535, 13921,
         2444,    90,   888,  2444,    90,   888,   141, 81753,   489,   457,
          239,   623,  2769,   261,  2191,  9926, 10916,     8, 75500,  3255,
         9566, 23535, 13921,  2507,   100,  3936,   569,     4,  8352,  8082,
         2507,  1759,   557,   569,     4,  6952,     6,  5967,     8,   115,
          898,   119,  3068,    21,  7129, 18324,  3758,    17,    25, 36101,
           53,   133, 10874,  3936,  5118,  1844,  1137,  8396,  5466,  2239,
         6630,   299,   285, 20754, 17050,   142,   218, 13729,  6657,  2414,
  

In [13]:
print(len(train_dataset))
print(len(test_dataset))

14094
3273


In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Linear(embed_dim, num_class)
        self.softmax = nn.Softmax(dim=1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        embedded = self.dropout(embedded)
        return self.softmax(self.fc(embedded))

In [16]:
VOCAB_SIZE = len(vocab)
EMBED_DIM = 300
NUN_CLASS = 4
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

def generate_batch(batch):
    label = torch.tensor([label_pipeline(entry[0]) for entry in batch])
    text = [torch.tensor(text_pipeline(entry[1]), dtype=torch.int64) for entry in batch]
    offsets = [0] + [entry.size(0) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [17]:
def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [ ]:
N_EPOCHS = 15
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.90)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

In [82]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0009(test)	|	Acc: 37.1%(test)


In [53]:
labels = {0: 'Q1', 1: 'Q2', 2: 'Q3', 3: 'Q4'}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        # text = torch.tensor([vocab[token]
        #                     for token in ngrams_iterator(tokenizer(text), ngrams)])
        text = torch.tensor(text_pipeline(text_preprocessing(text)))
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item()

vocab = vocab
model = model.to("cpu")

In [ ]:
ex_text_str = "What's it like in this dream called life? You and me It seems the question You'll leave out family to be free? It can't be that easy What's it like killing time in this dream called life? What's it like every night in this dream called life? So much to say, today Today, not the last, my endeavors change the past So much happened way to early She was girly One month has passed Afraid of coming clean, sensations but a dream, or so it seems What's it like killing time in this dream called life? What's it like every night in this dream called life? It's cold tonight Do you feel alone? Are you okay? Are you...okay? What's it like killing time in this dream called life? What's it like every night."

print("This is a %s news" %labels[predict(ex_text_str, model, vocab, NGRAMS)])

In [12]:
ex_text_str2 = """Day breaks and when I turn
You're not on my pillow
My head shakes and then the penny drops
Still your t-shirt on my chair
Penciled hearts on paper
I try not to think but I can't stop
And doubt sets in like a storm
I could feel it coming
And words echo out yesterday
Time pushed me to the edge
To jump was my decision
I've only got myself to blame
'Cause I have no right to love you
When I chose to walk away
I have no right to miss you
When I didn't wanna stay
And I have no right to need you
And I knew what my heart was gonna lose
I have no right to love you
But I do
I still do
I still do
Night falls, I'm wide awake
Wishing I could call you
I'm too used to hearing all about your day
The cold creeps up next to me
Now we don't share these covers
But I've no license to complain
'Cause I have no right to love you
When I chose to walk away
I have no right to miss you
When I didn't wanna stay
And I have no right to need you
And I knew what my heart was gonna lose
I have no right to love you
But I do
I still do
You know I care, but I can't be there
To make this unmade bed of roses
And you deserve to be put first
I had to let you go
I have no right to love you
When I chose to walk away
I have no right to miss you
When I didn't wanna stay
And I have no right to need you
And I knew what my heart was gonna lose
To ask if you're okay
When I left you so confused
I have no right to love you
I have no right, oh
I have no right to love you
But I still do
Yeah, I still do, whoa
I still do"""

print("This text belongs to %s class" %labels[predict(ex_text_str2, model, vocab, NGRAMS)])